In [5]:
import wincnn as wc
import numpy as np
import math
import os
LI = (0, 1, -1, 2, -2, 1/2, -1/2, 4, -4, 1/4, -1/4) 

Filter = (4, 1, 13, 9, 3, 12, 7, 12, 4, 1) 
Nadd = np.zeros((20, 20), dtype=int)

L = 10

#判斷是否為2的次方
def power_of_two(n):
    if isinstance(n, (int, float)):
        return math.log2(abs(n)).is_integer()
    else:
        return False

#將有小數點的數轉換成最小的整數，小數有效位數為4
def decimal_to_integer(value):
    smallest_value = 1
    while True:
        result = round(value,4) * smallest_value
        if result.is_integer():
            return int(result)
        smallest_value *= 2

#小數要乘上多少2次方才會變成整數
def power_time(value):
    smallest_value = 1
    while True:
        result = round(value,4) * smallest_value 
        if result.is_integer():
            return int(math.log2(smallest_value))
        smallest_value *= 2




In [6]:

x = decimal_to_integer(9.750003)

print(x)

39


In [7]:
# n input, r filter, m output
        
for n in range(4, 10, 1):
    a = LI[:n]
    #print(a)
    for r in range(3, 6, 1):
        m = n - r + 1
        if (m<2):
            break
        
        g = np.array(Filter[:r])

        
        f = wc.FdiagPlus1(a, n)
        if f[0,0] < 0:
            f[0,:] *= -1

        AT = np.array(wc.A(a, n, m).T, dtype = float).flatten()
        G = np.array( ( wc.A(a, n, r).T * f**(-1) ).T, dtype = float)
        BT = np.array( f * wc.B(a, n).T, dtype = float).flatten()
        h = np.round(np.matmul(G, g).flatten()).astype(float) #四捨五入整數
        
        os.makedirs('../win_verilog/wc_%d_%d/' %(m, r) , exist_ok=True)
        path = '../win_verilog/wc_%d_%d/' % (m, r)
        file = 'WC.v'
        outf = open(path + file, 'w')
        outf.write('// %s generated by WCSyn.py\n' % file)
        outf.write('`timescale 1ns / 1ps\n')

        outf.write('module WC(D, clk, rst, Z);\n')#D與BT相乘, G會導入已經乘好的h, 再將兩者相乘後乘上AT輸出為Z
        outf.write('input  [%d*%d-1:0] D;\n' % (L,n))
        outf.write('input  clk, rst;\n')
        #outf.write('input  [%d:0] G[%d:0];\n' % (L,n-1))
        outf.write('output reg [%d*%d-1:0] Z;\n\n' % (L,m))

        outf.write('\treg [%d:0] input_t [0:%d];\n' % (L-1, n-1))#存BT*d結果
        outf.write('\treg [%d:0] d [0:%d];\n' % (L-1, n*n-1))#準備做BT*d
        outf.write('\treg [%d:0] MUL [0:%d];\n' % (L-1, n-1))#準備(G*g)(BT*d)的相乘
        outf.write('\treg [%d:0] output_t [0:%d];\n\n' % (L-1, m-1))#最後乘上AT
        #outf.write('\tinteger i;\n')

        outf.write('\talways@(posedge clk)begin\n\n')
        #input transform
        outf.write('\t\t//input_transform\n')
        for i in range(0,n):
            for j in range(0,n):
                if BT[(n*n-1)-(j+i*n)] == 0:
                    outf.write('\t\td[%d] <= 0;\n' % ((n*n-1)-(j+i*n)))
                else:
                    if power_of_two(BT[(n*n-1)-(j+i*n)]):
                        if abs(BT[(n*n-1)-(j+i*n)]) > 1:
                            if BT[(n*n-1)-(j+i*n)] > 0:
                                outf.write('\t\td[%d] <= (D[%d:%d]) <<< %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, round(math.log2(abs(BT[(n*n-1)-(j+i*n)])))))
                            else:
                                outf.write('\t\td[%d] <= (~D[%d:%d] + 1) <<< %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, round(math.log2(abs(BT[(n*n-1)-(j+i*n)])))))
                        else:
                            if abs (BT[(n*n-1)-(j+i*n)]) == 1:
                                if BT[(n*n-1)-(j+i*n)] > 0:
                                    outf.write('\t\td[%d] <= (D[%d:%d]);\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10))
                                else:
                                    outf.write('\t\td[%d] <= (~D[%d:%d] + 1);\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10))                                
                            else:
                                if BT[(n*n-1)-(j+i*n)] > 0:
                                    outf.write('\t\td[%d] <= (D[%d:%d]) >>> %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, -round(math.log2(abs(BT[(n*n-1)-(j+i*n)])))))
                                else:
                                    outf.write('\t\td[%d] <= (~D[%d:%d] + 1) >>> %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, -round(math.log2(abs(BT[(n*n-1)-(j+i*n)])))))
                                
                    else:
                        if BT[(n*n-1)-(j+i*n)] > 0:
                            if BT[(n*n-1)-(j+i*n)] == int(BT[(n*n-1)-(j+i*n)]):
                                outf.write('\t\td[%d] <= (D[%d:%d]) * %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, BT[(n*n-1)-(j+i*n)]))
                            else:
                                outf.write('\t\td[%d] <= ((D[%d:%d]) * %d) >>> %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, int(decimal_to_integer(BT[(n*n-1)-(j+i*n)])), power_time(BT[(n*n-1)-(j+i*n)])))
                        else:
                            if BT[(n*n-1)-(j+i*n)] == int(BT[(n*n-1)-(j+i*n)]):
                                outf.write('\t\td[%d] <= (~D[%d:%d] + 1) * %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, -BT[(n*n-1)-(j+i*n)]))
                            else:
                                outf.write('\t\td[%d] <= ((~D[%d:%d] + 1) * %d) >>> %d;\n' % ((n*n-1)-(j+i*n), L*(j+1)-1, j*10, int(decimal_to_integer(-BT[(n*n-1)-(j+i*n)])), power_time(-BT[(n*n-1)-(j+i*n)])))
        outf.write('\n')


        for i in range(0,n):
            outf.write('\t\tinput_t[%d] <=' % (i))
            for j in range(0,n - 1):
                if BT[j+i*n] == 0:
                    outf.write('0 + ')
                else:
                    outf.write(' (d[%d]) + ' % (j+i*n))#input transform
            if BT[j+i*n+1] == 0:
                outf.write('0;\n')#打個零 不然會多一個+
            else:
                outf.write(' (d[%d]);\n'% (j+i*n+1))


        outf.write('\n')
        outf.write('\t\t//G*g\n')
        for i in range(0,n):
            if h[i] == 0:
                outf.write('\t\tMUL[%d] <= 0;\n' % (i))
            else:
                if power_of_two(h[i]):
                    if abs(h[i]) > 1:
                        if h[i] > 0:
                            outf.write('\t\tMUL[%d] <= (input_t[%d]) <<< %d;\n' % (i, i, round(math.log2(abs(h[i])))))
                        else:
                            outf.write('\t\tMUL[%d] <= (~input_t[%d] + 1) <<< %d;\n' % (i, i, round(math.log2(abs(h[i])))))
                    else:
                        if abs(h[i]) == 1:
                            if h[i] > 0:
                                outf.write('\t\tMUL[%d] <= (input_t[%d]);\n' % (i, i))
                            else:
                                outf.write('\t\tMUL[%d] <= (~input_t[%d] + 1);\n' % (i, i))                                
                        else:
                            if h[i] > 0:
                                outf.write('\t\tMUL[%d] <= (input_t[%d]) >>> %d;\n' % (i, i, -round(math.log2(abs(h[i])))))
                            else:
                                outf.write('\t\tMUL[%d] <= (~input_t[%d] + 1) >>> %d;\n' % (i, i, -round(math.log2(abs(h[i])))))
                                
                else:
                    if h[i] > 0:
                        outf.write('\t\tMUL[%d] <= (input_t[%d])*%d;\n' % (i, i, h[i]))
                    else:
                        outf.write('\t\tMUL[%d] <= (~input_t[%d] + 1)*%d;\n' % (i, i, -h[i]))#G*g
        outf.write('\n')
        
        for i in range(0,m):
            outf.write('\t\toutput_t[%d] <= ' % (i))
            for j in range(0,n - 1):
                if AT[j+i*n] == 0:
                    outf.write(' 0 + ')
                else:
                    if power_of_two(AT[j+i*n]):
                        if abs(AT[j+i*n]) > 1:
                            if AT[j+i*n+1] >= 0:
                                outf.write('(MUL[%d] <<< %d) + ' % (j, round(math.log2(abs(AT[j+i*n])))))
                            else:
                                outf.write('(MUL[%d] <<< %d) - ' % (j, round(math.log2(abs(AT[j+i*n])))))
                        else:
                            if abs(AT[j+i*n]) == 1:
                                if AT[j+i*n+1] >= 0:
                                    outf.write('(MUL[%d]) + ' % (j))
                                else:
                                    outf.write('(MUL[%d]) - ' % (j))
                                
                            else:
                                if AT[j+i*n+1] >= 0:
                                    outf.write('(MUL[%d] >>> %d) + ' % (j, -round(math.log2(abs(AT[j+i*n])))))
                                else:
                                    outf.write('(MUL[%d] >>> %d) - ' % (j, -round(math.log2(abs(AT[j+i*n])))))
                                
                   # outf.write(' (MUL[%d]) * %f + ' % (j, AT[j+i*n]))
            if AT[j+i*n+1] == 0:
                outf.write('0;\n')
            else:
                outf.write(' (MUL[%d]);\n' % (j+1,))
        outf.write('\n')


        for i in range(0,m):
            outf.write('\t\tZ[%d:%d] <= output_t[%d];\n' % ((m-i)*L-1, (m-i-1)*L, i))
        outf.write('\n')
        outf.write('\tend\n')
        '''
        outf.write('\tgenvar j;\n')
        outf.write('\tgenerate\n')
        outf.write('\tfor(j=0; j<%d; j=j+1)\n' % (m))
        outf.write('\t\tbegin:output_block\n')
        outf.write('\t\t\tassign Z[(%d+%d*j):(0+%d*j)] = output_t[%d-j];\n' % (L-1, L, L, m-1))
        outf.write('\t\tend\n')
        outf.write('\tendgenerate\n')
        '''


        outf.write('endmodule\n')
        outf.close()
        print('wc_%d_%d.v created in ./verrilog' % (m, r))



wc_2_3.v created in ./verrilog
wc_3_3.v created in ./verrilog
wc_2_4.v created in ./verrilog
wc_4_3.v created in ./verrilog
wc_3_4.v created in ./verrilog
wc_2_5.v created in ./verrilog
wc_5_3.v created in ./verrilog
wc_4_4.v created in ./verrilog
wc_3_5.v created in ./verrilog
wc_6_3.v created in ./verrilog
wc_5_4.v created in ./verrilog
wc_4_5.v created in ./verrilog
wc_7_3.v created in ./verrilog
wc_6_4.v created in ./verrilog
wc_5_5.v created in ./verrilog


In [ ]:
x = decimal_to_integer(BT[39])
x


-5488762045857791